In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean_datav4.csv


In [2]:
df_orig = pd.read_csv("/kaggle/input/clean_datav4.csv")

In [3]:
df_orig.head()

,Unnamed: 0,Unnamed: 0.1,ID,tweet_id,text,missing_text,Text_Only_Informative,Image_Only_Informative,Directed_Hate,Generalized_Hate,Sarcasm,Allegation,Justification,Refutation,Support,Oppose
0,0,0,0,1.052240e+18,new post domestic violence awareness caught me...,0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1,1,1,1.052210e+18,domestic violence awareness caught metoo,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,2,2,1.052180e+18,mother nature metoo,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,3,1.052160e+18,ption no2,0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,4,4,4,1.052100e+18,high time metoo named shamed men medium advert...,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [4]:
import tensorflow as tf

In [5]:
!pip3 install bert-for-tf2

     |████████████████████████████████| 40 kB 2.7 MB/s eta 0:00:011
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-py3-none-any.whl size=30114 sha256=051f29c8000fb6b103ad9e3b0524fdee5d6fb6795d5f56860130f4a9f7b612ac
  Stored in directory: /root/.cache/pip/wheels/6c/c9/9c/363182ea34a736dae336eeaf0dd4a7eec3c6a5afe32373e1fe
  Created wheel for py-params: filename=py_params-0.9.7-py3-none-any.whl size=7302 sha256=68d1e546e62917456167198b538779cd05432c9da3495d10c01a6ff611e310ac
  Stored in directory: /root/.cache/pip/wheels/47/3d/2d/bbffcfd6b9f4b8b5cbf07e7520ac2676192fe9431240c13ee8
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19473 sha256=20d0704a9d8b2044d932423255fd571eb29d4846fe9c47386e3daaecf930c614
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
Successfully built bert-for-tf2 py-params params-flow


In [6]:
import tensorflow_hub as hub 
from bert import bert_tokenization
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [7]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [9]:
df_orig['text']=df_orig['text'].astype(str)

In [10]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    net = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(1, activation='sigmoid')(net)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [11]:
max_len = 100

In [12]:
from sklearn import model_selection

In [13]:
X_train,X_val,y_train,y_val = model_selection.train_test_split(df_orig['text'],df_orig['Directed_Hate'],test_size = 0.2,random_state = 4)

In [14]:
X_train = bert_encode(X_train, tokenizer, max_len=max_len)
X_val = bert_encode(X_val, tokenizer, max_len=max_len)

In [15]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [16]:
%%time
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_history = model.fit(
    X_train, y_train, 
   
    epochs=10,
    callbacks=[checkpoint, earlystopping],
    batch_size=1,
    verbose=1
)

Epoch 1/10
6382/6382 [==============================] - 255s 40ms/step - loss: 0.1883 - accuracy: 0.9627
Epoch 2/10
6382/6382 [==============================] - 253s 40ms/step - loss: 0.1789 - accuracy: 0.9627
Epoch 3/10
6382/6382 [==============================] - 254s 40ms/step - loss: 0.1725 - accuracy: 0.9627
Epoch 4/10
6382/6382 [==============================] - 253s 40ms/step - loss: 0.1733 - accuracy: 0.9629
Epoch 5/10
6382/6382 [==============================] - 253s 40ms/step - loss: 0.1754 - accuracy: 0.9629
Epoch 6/10
6382/6382 [==============================] - 253s 40ms/step - loss: 0.1731 - accuracy: 0.9629
Epoch 7/10
6382/6382 [==============================] - 255s 40ms/step - loss: 0.1648 - accuracy: 0.9622
Epoch 8/10
6382/6382 [==============================] - 252s 40ms/step - loss: 0.1389 - accuracy: 0.9632
Epoch 9/10
6382/6382 [==============================] - 253s 40ms/step - loss: 0.1231 - accuracy: 0.9611
Epoch 10/10
6382/6382 [==============================] 

In [19]:
pr = model.evaluate(X_val,y_val)

print("Loss: " + str(pr[0]))
print("Accuracy: " + str(pr[1]*100))

50/50 [==============================] - 6s 123ms/step - loss: 0.2910 - accuracy: 0.9549
Loss: 0.2910076081752777
Accuracy: 95.48872113227844
